<a href="https://colab.research.google.com/github/saralieber/CS_Studio/blob/master/Review_Ch7_Part2_Text_to_Vectors_Meanings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deriving meaning from words

Is there a way to represent all English words so they have this "closer in space is closer in meaning" property that we saw with colors represented as their RGB properties?

To answer this, we have to first think of what *meaning* means.

One theory (Distributional Hypothesis) popular among computational linguists is that linguistic items with similar contexts have similar meanings.

In other words, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

How do we turn this insight into a system for creating general-purpose vectors that capture the meaning of words?

Let's use a small source text to begin with, such as this except from Dickens:

    It was the best of times, it was the worst of times.

This spreadsheet tries to capture the context of words. 
![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]

You could use the same distance formula we defined before to get useful information about which vectors in this space are similar to each other.

In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the __ of`).

Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

In many texts, there will be many thousands if not millions of possible contexts. It turns out, though, that many of these dimensions (contexts) will end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors.

The process of getting rid of superfluous dimensions in a vector space is called *dimensionality reduction*.

The question of how to identify a "context" is difficult to answer. 

You might want to...

*   Use the word before and after the given word (e.g., see example above)
*   Use a larger window (e.g., the two words before and after the given word)
*   Use a non-contiguous window (e.g., skip a word before and after the given word)
*   Look at larger syntactic structure: what are the syntactic-contexts you find the word in?
*   Exclude certain "function" words like "the" and "of" 
*   Lemmatize the words before you begin your analysis so two occurences with different "forms" of the same word count as the same context

These are all questions open to research and debate.




## GloVe vectors

You don't have to create your own word vectors from scratch. Many researchers have made downloadable databases of pre-trained vectors.

One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll use for this activity. They come with `en_core_web_md`.

In [0]:
import spacy
!python -m spacy download en_core_web_md # download the dictionary
import en_core_web_md
nlp = en_core_web_md.load()

In [0]:
nlp.vocab.has_vector('frankenstein') # Check to make sure word vectors have been loaded

True

In [0]:
dogv = nlp.vocab['dog'].vector # get the 300-dimensional vector for dog

In [0]:
type(dogv)

numpy.ndarray

In [0]:
dog_list = dogv.tolist()

In [0]:
len(dog_list) # 300

300

In [0]:
dog_list[:10]

[-0.4017600119113922,
 0.37057000398635864,
 0.02128100022673607,
 -0.3412500023841858,
 0.04953800141811371,
 0.29440000653266907,
 -0.17375999689102173,
 -0.2798199951648712,
 0.06762199848890305,
 2.169300079345703]

In [0]:
# The following function gets a vector of a given string from spaCy's vocabulary

def get_vec(s:str) -> list:
  return nlp.vocab[s].vector.tolist()

In [0]:
get_vec('dog') == dog_list # should be True

True

In [0]:
# There is also a vector for words not in the vocab. It is all zeroes.

zero_vec = get_vec('askfsda') # Not in vocab
zero_vec.count(0) # 300 zeroes, i.e., all zeroes

300

In [0]:
# Import uo_puddles library

!rm -r 'uo_puddles'
my_github_name = 'uo-puddles' # can replace with your account name
clone_url = f'https://github.com/{my_github_name}/uo_puddles.git'
!git clone $clone_url
import uo_puddles.uo_puddles as up

rm: cannot remove 'uo_puddles': No such file or directory
Cloning into 'uo_puddles'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 234 (delta 139), reused 64 (delta 33), pack-reused 0
Receiving objects: 100% (234/234), 59.83 KiB | 2.99 MiB/s, done.
Resolving deltas: 100% (139/139), done.


In [0]:
# The following shows that cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`

up.cosine_similarity(get_vec('dog'), get_vec('puppy')) > up.cosine_similarity(get_vec('trousers'), get_vec('octopus'))

True

## Sentence Similarity

Moving toward doing prediction. We will be interested in converting an entire sentence into a single glove vector.

Here is the general idea of how we will do this. First, go through each row and grab the text of the sentence. Build a list of guarded tokens (kind of like e_list for Naive Bayes).

Then, get the vectors for all the tokens and build a matrix. Take the average. You can use that vector average as the representation of the sentence.

In [0]:
# Example

pilot_sentences = [
  'It was really cold yesterday.',
  'It will be really warm today, though.',
  "It'll be really hot tomorrow!'",
  'Will it be really cool Tuesday?'
]

In [0]:
def addv(x:list, y:list) -> list: # define a function, called addv, that takes variables x & y (both lists) as inputs
  assert isinstance(x, list), f"x must be a list but instead is {type(x)}"
  assert isinstance(y, list), f"y must be a list but instead is {type(y)}"
  assert len(x) == len(y), f"x and y must be the same length"

  #result = [(c1 - c2) for c1, c2 in zip(x, y)]  #one-line compact version - called a list comprehension

  result = []
  for i in range(len(x)):
    c1 = x[i]
    c2 = y[i]
    result.append(c1+c2)

  return result

def dividev(x:list, y:int) -> list:
  assert isinstance(x, list), f"x must be a list but instead is {type(x)}"
  assert isinstance(y, int), f"y must be an integer but instead is {type(y)}"

  result = []
  for i in range(len(x)): 
    c1 = x[i]
    result.append(c1/y)
  return result 

def meanv(matrix:list) -> list:
  assert isinstance(matrix, list), f"matrix must be a list but instead is {type(x)}"
  assert len(matrix) >=1, f"matrix must have at least one row"

  sumv = matrix[0] # start with the first row
  for row in matrix[1:]: # add each row to the first row, starting with the second row
    sumv = addv(sumv, row) # take the sum of the first+second row, and then this resulting sum plus the third row
  mean = dividev(sumv, len(matrix)) # divide the sum of all the rows by the number of rows
  return mean

In [0]:
# Get the average vector for the first (0th) sentence in pilot_sentences. Store it in s0_average. Should be of length 300.

first_sent = pilot_sentences[0]
doc = nlp(first_sent.lower())

vectors = []

for token in doc:
  word = token.text
  vecs = get_vec(word)
  vectors.append(vecs)

s0_average = meanv(vectors)

300

In [0]:
len(s0_average) # 300

In [0]:
print(s0_average[:10])

[0.02954764893123259, 0.32462666432062787, -0.013579833010832468, -0.2010739967226982, -0.03903430079420408, 0.014894500374794006, -0.01435499886671702, -0.27804166823625565, -0.10192649935682614, 2.4969166914621987]
